In [1]:
!pip install -q transformers datasets evaluate accelerate torch bitsandbytes pandas scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
    TrainingArguments, Trainer, pipeline
)
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

teacher_model_name = "xlm-roberta-base"
student_model_name = "roberta-base"
num_labels = 2  # positive / negative


2025-10-23 16:40:53.116082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761237653.296281      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761237653.349328      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Device: cuda


In [3]:
import os, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
    TrainingArguments, Trainer, pipeline
)
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

teacher_model_name = "xlm-roberta-base"
student_model_name = "roberta-base"
num_labels = 2  # positive / negative


✅ Device: cuda


In [4]:
# Paths to your txt files
pos_path = "/kaggle/input/dataaaaaa/all_positive_8500.txt"
neg_path = "/kaggle/input/dataaaaaa/all_negative_3307.txt"

# Read files (one comment per line)
with open(pos_path, 'r', encoding='utf-8') as f:
    pos_lines = [line.strip() for line in f if line.strip()]

with open(neg_path, 'r', encoding='utf-8') as f:
    neg_lines = [line.strip() for line in f if line.strip()]

# Create DataFrame
df_pos = pd.DataFrame({"text": pos_lines, "label": 1})
df_neg = pd.DataFrame({"text": neg_lines, "label": 0})
df = pd.concat([df_pos, df_neg], ignore_index=True)
print("✅ Dataset shape:", df.shape)
print(df.head())

# Save merged CSV (optional)
csv_path = "bengali_youtube_sentiment.csv"
df.to_csv(csv_path, index=False)
print(f"💾 Saved merged dataset to {csv_path}")


✅ Dataset shape: (11807, 2)
                                                text  label
0                    অসাধারণ নিশো বস্ আর অমি ভাইকেও।      1
1                               আমার দেখা বেস্ট নাটক      1
2  নাটক টা অনেক সুন্দর হয়েছে,,,,আফরান নিশো ভাইয়...      1
3                          সত্যি অসাধারণ একটি রিলেশন      1
4                     মজা পাইছি ভাষা গুলো কেমন লাগলো      1
💾 Saved merged dataset to bengali_youtube_sentiment.csv


In [5]:
# Train 80%, Val 10%, Test 10%
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

# Convert to Hugging Face DatasetDict
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})
dataset


Train: 9445 Val: 1181 Test: 1181


DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 9445
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1181
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 1181
    })
})

In [6]:
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name)

def preprocess(batch):
    return student_tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized_ds = dataset.map(preprocess, batched=True)
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

print(tokenized_ds)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/9445 [00:00<?, ? examples/s]

Map:   0%|          | 0/1181 [00:00<?, ? examples/s]

Map:   0%|          | 0/1181 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 9445
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1181
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1181
    })
})


In [7]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

teacher_model = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_name, num_labels=num_labels
).to(device)

teacher_args = TrainingArguments(
    output_dir="./xlmr_teacher_youtube",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs_teacher",
    report_to="none"
)

trainer_teacher = Trainer(
    model=teacher_model,
    args=teacher_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics,
)

trainer_teacher.train()


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_37/2311260714.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_teacher = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.423243,0.802710
2,0.487700,0.394028,0.846740
3,0.487700,0.318604,0.854361
4,0.335300,0.330422,0.861981
5,0.335300,0.317992,0.865368


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1480, training_loss=0.3699384843980944, metrics={'train_runtime': 1086.1938, 'train_samples_per_second': 43.478, 'train_steps_per_second': 1.363, 'total_flos': 3106354897344000.0, 'train_loss': 0.3699384843980944, 'epoch': 5.0})

In [8]:
teacher_results = trainer_teacher.evaluate(tokenized_ds["test"])
print("📊 Teacher Accuracy:", teacher_results)

teacher_path = "./finetuned_xlmr_youtube"
trainer_teacher.save_model(teacher_path)
teacher_tokenizer.save_pretrained(teacher_path)


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


📊 Teacher Accuracy: {'eval_loss': 0.305649995803833, 'eval_accuracy': 0.8831498729889924, 'eval_runtime': 8.2125, 'eval_samples_per_second': 143.806, 'eval_steps_per_second': 4.505, 'epoch': 5.0}


('./finetuned_xlmr_youtube/tokenizer_config.json',
 './finetuned_xlmr_youtube/special_tokens_map.json',
 './finetuned_xlmr_youtube/sentencepiece.bpe.model',
 './finetuned_xlmr_youtube/added_tokens.json',
 './finetuned_xlmr_youtube/tokenizer.json')

In [9]:
import shutil
shutil.make_archive("finetuned_xlmr_youtube", 'zip', "./finetuned_xlmr_youtube")


'/kaggle/working/finetuned_xlmr_youtube.zip'